# Equity OTC Binary Option Samples

This notebook demonstrates how to access and use the financial-contracts end point of Instrument Pricing Analytics (IPA) in the Refinitiv Data Platform.

##### Learn more

Please visit the [Refinitiv Developer Community Portal](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-platform-apis/documentation#ipa-financial-contracts-option-contracts-eti) to access the API documentation and much more.

You may also visit the [API Playground](https://apidocs.refinitiv.com/Apps/ApiDocs) to get more examples, find  end points and download Swagger files

##### Getting Help and Support

If you have any questions regarding the API usage, please post them on the [Refinitiv Data Platform Q&A Forum](https://community.developers.refinitiv.com/spaces/231/index.html). The Refinitiv Developer Community will be very pleased to help you. 

#### Import the library and connect to the Refinitv Data Platform

This sample works in both desktop and non-desktop environment. The appropriate session can be created based on the constant variable IPA_ENVIRONMENT, that is set by default to DESKTOP for execution in the Refinitiv codebook application. To run this sample in a non-desktop environment, you may set the variable to NON-DESKTOP.

In [1]:
import refinitiv.dataplatform as rdp

# Set IPA_ENVIRONMENT to NON-DESKTOP to use this example in a non desktop environment
# IPA_ENVIRONMENT = "NON-DESKTOP"
IPA_ENVIRONMENT = "DESKTOP"

if (IPA_ENVIRONMENT == "DEKSTOP"):
    session = rdp.DesktopSession(app_key='DEFAULT_CODE_BOOK_APP_KEY')
else:
    %run ".\..\..\credentials.ipynb"
    session = rdp.PlatformSession(APP_KEY, rdp.GrantPassword(RDP_LOGIN, RDP_PASSWORD))

session.open()
endpoint = rdp.Endpoint(session, "https://api.edp.thomsonreuters.com/data/quantitative-analytics/v1/financial-contracts")
print("session opened and end-point set")

session opened and end-point set


### Valuing an OTC Binary Option

This sample value an existing vanilla option from the option definition provided in the request. The returned 'fields' are:
- the market value in cash (MarketValueInDealCcy,)
- the main Greeks (DeltaPercent, GammaPercent, RhoPercent)
- the error information (ErrorCode,ErrorMessage)

the full documentation of the parameters available for equity options can be found in the [Developer Community Portal](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-platform-apis/documentation#ipa-financial-contracts-option-contracts-eti)


In [13]:
request_body = {
    
    "fields" : ["InstrumentTag","MarketValueInDealCcy", "DeltaPercent", "GammaPercent", "RhoPercent", "ErrorCode","ErrorMessage"],
    
    "universe":[{
        "instrumentType": "Option",
        "instrumentDefinition": {
            "underlyingType": "Eti",
            "strike": 50,
            "endDate": "2022-01-15",
            "callPut": "Call",
            "underlyingDefinition": {
                "instrumentCode": "RENA.PA"
            },
            "BinaryDefinition":{
                "binaryType":"NoTouch",
                "upOrDown":"Down",
                "level":30,
                "notionalAmount":100
            }
        }
    }],     
    
    "pricingParameters": {
        "valuationDate": "2021-01-12T00:00:00Z",
        "volatilityType":"SVISurface"
    },
        
    "outputs" : ["Data","Headers"],

}

response = endpoint.send_request(
    method = rdp.Endpoint.RequestMethod.POST,
    body_parameters = request_body)

print(response.data.raw)

{'headers': [{'type': 'String', 'name': 'InstrumentTag'}, {'type': 'Float', 'name': 'MarketValueInDealCcy'}, {'type': 'Float', 'name': 'DeltaPercent'}, {'type': 'Float', 'name': 'GammaPercent'}, {'type': 'Float', 'name': 'RhoPercent'}, {'type': 'String', 'name': 'ErrorCode'}, {'type': 'String', 'name': 'ErrorMessage'}], 'data': [[None, 69.8371284666669, -3.9714440077309, 0.137248061950846, -0.942949129902136, '', '']]}


In [14]:
import pandas as pd

headers_name = [h['name'] for h in response.data.raw['headers']]
df = pd.DataFrame(data=response.data.raw['data'], columns=headers_name)
df

,InstrumentTag,MarketValueInDealCcy,DeltaPercent,GammaPercent,RhoPercent,ErrorCode,ErrorMessage
0,None,69.837128,-3.971444,0.137248,-0.942949,,


2021-01-12 16:51:44,577 - Session session.platform - Thread 13924 | MainThread
[Error 400 - invalid_grant] empty error description
